## Libraries

In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

## Function

In [2]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

In [3]:
import matrix_eig as me

## Train monster

### Constant parameters

In [4]:
data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig)),
               ('spectral', Transformer(spectral_norm))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low5', Transformer(me.matrix_eig_5, collect=['X_vec'])),
               ('low10', Transformer(me.matrix_eig_10, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low30', Transformer(me.matrix_eig_30, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low50', Transformer(me.matrix_eig_50, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low70', Transformer(me.matrix_eig_70, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low90', Transformer(me.matrix_eig_90, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low110', Transformer(me.matrix_eig_110, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low130', Transformer(me.matrix_eig_130, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low150', Transformer(me.matrix_eig_150, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low170', Transformer(me.matrix_eig_170, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low190', Transformer(me.matrix_eig_190, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low210', Transformer(me.matrix_eig_210, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR', LogisticRegression())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     LR=dict(
        C=[0.1*i for i in range(1, 11)],
        max_iter=[50],
        penalty=['l1', 'l2']
    )
    )

### Variable parameters

In [5]:
num_epoch = 10

banned_combos = [('LR', 'origS'),
                 ('LR', 'origN')]

list_of_files = ["results_1.csv", "results_2.csv", "results_3.csv", "results_4.csv", "results_5.csv", 
                 "results_6.csv", "results_7.csv", "results_8.csv", "results_9.csv", "results_10.csv"]

In [6]:
for epoch in range(num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=1)

    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Orig-Spectral_ofTheBest/" + list_of_files[epoch])

No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30
No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 13/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/3

In [13]:
weighters = [('binar', Transformer(orig))]

banned_combos = [('LR', 'origS'),
                 ('LR', 'spectral')]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

In [15]:
for epoch in range(num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=1)

    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Binar-Orig_ofTheBest/" + list_of_files[epoch])

Removed previous results file -- Binar-Orig_ofTheBest/results_1.csv.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30


Process PoolWorker-9804:
    self._target(*self._args, **self._kwargs)
Process PoolWorker-9803:
Traceback (most recent call last):
Process PoolWorker-9798:
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-9802:
    self.run()
Process PoolWorker-9800:
Traceback (most recent call last):
Process PoolWorker-9801:
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
Process PoolWorker-9799:
Process PoolWorker-9806:
Process PoolWorker-9805:
Traceback (most recent call last):
Process PoolWorker-9808:
    task = get()
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _boots

KeyboardInterrupt: 